In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from  scipy.stats import skew, kurtosis, shapiro

# Import the data
* The data contains Na values in every row of the csv file

In [4]:
consumptionLoad = pd.read_csv("D:\Onedrive\Leuven\Final project\data\consumption.csv",index_col = "meter_id")
new_index = np.arange(1,len(consumptionLoad.index)+1)
consumptionLoad.set_index(new_index,inplace=True)
consumptionLoad.head()

In [5]:
consumptionLoad_T = consumptionLoad.transpose()

In [ ]:
for i in range(len(consumptionLoad.index)) :
    print("Amount of Nan for meter ", i , " : " ,  consumptionLoad.iloc[i].isnull().sum())
pd.

stijn is staring
stijn is cool